*Okay. Alright. That's fine.*

\- Drake



Hyperparameter optimization did not as expected, but we have more exciting things ahead of us: creating our own graph classifier. Let's approach this task using PyTorch Geometric using this [example](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=mHSP6-RBOqCE) as reference.

We can break this down into a few steps:
- Convert SMILES data into graph data
- Mini-batch graph data
- Define Graph Neural Network for graph classification
- Train model and evaluate - you know the drill 

Let's get to it 🤖

In [4]:
!pip install deepchem
!pip install 'deepchem[torch]'
!pip install rdkit
!pip install torch_geometric


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv', reload=False)
train_dataset, valid_dataset, test_dataset = datasets

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading some Jax models, missing a dependency. No module named 'jax'
[23:36:34] WARNING: not removing hydrogen atom without neighbors
[23:36:46] WARNING: not removing hydrogen atom without neighbors


Let's take a close look at this dataset by inspecting the training dataset.

In [3]:
train_dataset

<DiskDataset X.shape: (6264,), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>

It looks like there are **6264** graphs (molecules) in our training dataset, where *X* stores the molecules, *y* stores the hot-encoded output vector with one entry for each of the **12** measures/tasks, and a weights matrix *w*.

Here's one particular molecule:

In [4]:
test_mol = train_dataset.X[0]
print(f"This molecule has {test_mol.n_atoms} atoms with {test_mol.n_feat} features each.")

This molecule has 11 atoms with 75 features each.


In [6]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(6264, 783, 784)

Note above sizes of our training, validation, and test datasets. However, we can't just operate on the above data directly as molecular information is stored as a [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string (that is, Simplified molecular-input line-entry system). Try saying that five times fast. In short, the specification enables structural information of molecules to be encoded into a string.

For example,

In [5]:
train_dataset.ids[0]

'CC(O)(P(=O)(O)O)P(=O)(O)O'

Fortunately, open-source is once again our savior: [RDKit](https://www.rdkit.org/) supplies a few helper functions to convert sparse adjacency matrices  into graph data that we *can* use with PyTorch Geometric. From there, we can extract additional metadata to help understand the numbers.

In [71]:
import scipy.sparse
import numpy as np
def adjacency_list_to_sparse(adj_list):
    num_nodes = len(adj_list)
    rows, cols = [], []

    for i, neighbors in enumerate(adj_list):
        rows.extend([i] * len(neighbors))
        cols.extend(neighbors)

    adjacency_matrix = scipy.sparse.coo_matrix((np.ones_like(rows), (rows, cols)),
                                              shape=(num_nodes, num_nodes),
                                              dtype=np.float32)

    return adjacency_matrix


In [131]:
import torch
from torch_geometric.data import Data
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.utils.convert import from_scipy_sparse_matrix


def to_data(mol_graph):

    x = mol_graph.get_atom_features()

    adj_list = mol_graph.get_adjacency_list()
    sparse_mat = adjacency_list_to_sparse(adj_list)
    edge_index, edge_attr = from_scipy_sparse_matrix(sparse_mat)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

If you are anything like me 15 minutes ago, you may be underwhelmed and/or perplexed by the above numbers. A little clarification on the `Data` object might help:

1. **x (Node Feature Matrix):**
   - This parameter represents the feature matrix for each node in the graph.
   - It is a PyTorch tensor with shape [num_nodes, num_node_features].
   - Each row corresponds to a node, and each column corresponds to a feature of that node.
   - For example, if you are representing atoms in a molecule, `x` could contain features like atomic number, charge, etc.


2. **edge_index (Graph Connectivity):**
   - `edge_index` represents the graph connectivity in COO (Coordinate List) format.
   - It is a PyTorch tensor with shape [2, num_edges].
   - Each column of `edge_index` contains the indices of two nodes that form an edge.
   - For an undirected graph, (i, j) and (j, i) should both be present in the `edge_index`.


3. **edge_attr (Edge Feature Matrix):**
   - `edge_attr` represents the feature matrix for each edge in the graph.
   - It is a PyTorch tensor with shape [num_edges, num_edge_features].
   - Each row corresponds to an edge, and each column corresponds to a feature of that edge.
   - This is often used to store information like bond types, distances, or any other edge-specific features.

While there are a few other parameters, these three collectively provide a comprehensive representation of the graph needed for our specific use case.

Let's convert all our datasets from SMILES to graph data.

In [141]:
train_dataset_graph = []
for mol_graph in train_dataset.X:
    data = to_data(mol_graph)
    train_dataset_graph.append(data)

valid_dataset_graph = []
for mol_graph in valid_dataset.X:
    data = to_data(mol_graph)
    valid_dataset_graph.append(data)

test_dataset_graph = []
for mol_graph in test_dataset.X:
    data = to_data(mol_graph)
    test_dataset_graph.append(data)

In [142]:
train_dataset_graph[0]

Data(x=[11, 75], edge_index=[2, 20], edge_attr=[20])

Convert SMILES data into graph data ✅



In [143]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_dataset_graph, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset_graph, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [146]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2226], edge_attr=[2226], batch=[1095], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2236], edge_attr=[2236], batch=[1074], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 1740], edge_attr=[1740], batch=[873], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2454], edge_attr=[2454], batch=[1189], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 1896], edge_attr=[1896], batch=[956], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2234], edge_attr=[2234], batch=[1094], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2046], edge_attr=[2046], batch=[1008], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[64], edg

Hmm. Slight issue with the x parameter, but no clear solution is popping out to me at the moment. As my friend Dan likes to say, "Let's circle back to this."

Also, small (very consequential) update: after spending 2 days trying to get perfectly preprocess this data, I have discovered that PyTorch Geometric supplies its own MoleculeNet class, complete with a Tox 21 dataset 🥲.

Despite how much my RAM has suffered due to tabs upon tabs of documentation, I'd say that this process helped clarify **how** and **why** we are preprocessing our data. Now let's condense yesterday's work into two lines!

In [147]:
from torch_geometric.datasets import MoleculeNet
dataset = MoleculeNet(root="./data/", name="Tox21")
dataset.data

/Users/mihirarya/Dev/bcil/tox21/env/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[145459, 9], edge_index=[2, 302190], edge_attr=[302190, 3], smiles=[7831], y=[7831, 12])

Its beautiful. We can finally move on...

Next step: **Mini-batching**.

Instead of "stacking" equally-sized matrices into a single mini-batch, as we may have done with image data, we take an alternative approach with graph data. "Why overcomplicate things?" you may ask. According to PyTorch Geometric [documentation](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=0gZ-l0npPIca):

1. GNN operators that rely on a **message passing scheme** (more on this later) do not need to be modified since messages are not exchanged between two nodes that belong to different graphs

2. There is no computational or memory overhead since adjacency matrices are saved in a sparse fashion holding only non-zero entries (*i.e.*, the edges)

PyTorch Geometric automatically takes care of **batching multiple graphs into a single giant graph** with the help of the [`torch_geometric.data.DataLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.DataLoader) class:

Mini-batch data ✅

Next step: Graph. Neural. Networks.

FINALLY. Let's make like Merriam-Webster and define what our GNN is going to look like.

But before we that, let's quickly review message-passing (you're welcome, future me).

In [26]:
train_dataset_graph[0].num_node_features

1

In [ ]:
from torch_geometric.nn import Sequential, GraphConv
num_edge_features = train_dataset_graph[0].num_edge_features

class GraphClassifier(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphClassifier, self).__init__()
        self.conv1 = GraphConv(num_edge_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, 